In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Dropping ID columns, 'EIN' and 'NAME'
application_df = application_df.drop(columns=["EIN", "NAME"])
#dropping SPECIAL_CONSIDERATIONS collum
application_df = application_df.drop(columns=["SPECIAL_CONSIDERATIONS"])


In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_counts = application_df["APPLICATION_TYPE"].value_counts()
print(app_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Chosing a cutoff value and creating a list of application types to be replaced

#increasing the cutoff value to 1000 instead of 500
application_types_to_replace = list(app_counts[app_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
Other,2266
T4,1542
T6,1216
T5,1173
T19,1065


In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
#  converting categorical data to numeric with `pd.get_dummies`

application_df = pd.get_dummies(application_df)

In [8]:
# Split our preprocessed data into our features and target arrays
#  Getting target variable
y = application_df["IS_SUCCESSFUL"]
# getting features
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#using MinMaxScaler instead of StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Scale the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model


In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#increasing neurons and using LeakyReLU instead of ReLU , Adding addiltional layers

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=256, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer (More neurons)
nn.add(tf.keras.layers.Dense(units=128, activation="relu"))

# Third hidden layer (More layers)
nn.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Fourth hidden layer (Try LeakyReLU)
nn.add(tf.keras.layers.Dense(units=32, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                      │ (None, 256)                 │          26,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 69,889 (273.00 KB)

 Trainable params: 69,889 (273.00 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Compile the model
#compiling the model
nn.compile(loss="binary_crossentropy", optimizer="adamax", metrics=["accuracy"])

In [21]:
#Creating a callback that saves the model's weights every five epochs
# Importing Keras callbacks
from tensorflow.keras.callbacks import ModelCheckpoint

# Defining a callback to save the model every 5 epochs
checkpoint_callback = ModelCheckpoint(filepath="weights.epoch_{epoch:02d}.weights.h5", save_freq=5 * (len(X_train_scaled) // 32), save_weights_only=True, verbose=1)

In [22]:
# Training the model and increasing the epochs
nn.fit(X_train_scaled, y_train, epochs=250, batch_size=32, callbacks=[checkpoint_callback])

Epoch 1/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7413 - loss: 0.5281
Epoch 2/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7474 - loss: 0.5222
Epoch 3/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7444 - loss: 0.5218
Epoch 4/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7439 - loss: 0.5252
Epoch 5/250
842/858 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7378 - loss: 0.5315
Epoch 5: saving model to weights.epoch_05.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7380 - loss: 0.5314
Epoch 6/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7460 - loss: 0.5253
Epoch 7/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7458 - loss: 0.5220
Epoch 8/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7435 - loss: 0.5257
Epoch 9/250
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7466 - loss: 0.5251
Epoch 10/250
842/858 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7428 - loss: 0.5259
E

In [23]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 3ms/step - accuracy: 0.7302 - loss: 0.9165
Loss: 0.9164903163909912, Accuracy: 0.7301748991012573


In [ ]:
# adjusting feature selection
application_df = application_df.drop(columns=["SPECIAL_CONSIDERATIONS", "AFFILIATION", "USE_CASE"])

In [24]:
# Define an optimized neural network model
nn_optimized = tf.keras.models.Sequential()

# First hidden layer (More neurons)
nn_optimized.add(tf.keras.layers.Dense(units=512, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer (More neurons)
nn_optimized.add(tf.keras.layers.Dense(units=256, activation="relu"))

# Third hidden layer (Use Swish activation)
nn_optimized.add(tf.keras.layers.Dense(units=128, activation="swish"))

# Fourth hidden layer (Keep LeakyReLU)
nn_optimized.add(tf.keras.layers.Dense(units=64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the optimized model
nn_optimized.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

# View summary
nn_optimized.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 512)                 │          53,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,793 (882.00 KB)

 Trainable params: 225,793 (882.00 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Learning rate scheduler
lr_callback = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [27]:
nn_optimized.fit(X_train_scaled, y_train, epochs=250, batch_size=16, callbacks=[checkpoint_callback, lr_callback])

Epoch 1/250
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.7166 - loss: 0.5763 - learning_rate: 0.0010
Epoch 2/250
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7318 - loss: 0.5524 - learning_rate: 0.0010
Epoch 3/250
 597/1715 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7245 - loss: 0.5528
Epoch 3: saving model to weights.epoch_03.weights.h5
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7286 - loss: 0.5527 - learning_rate: 0.0010
Epoch 4/250
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.7315 - loss: 0.5483 - learning_rate: 0.0010
Epoch 5/250
1458/1715 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7313 - loss: 0.5515
Epoch 5: saving model to weights.epoch_05.weights.h5
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7319 - loss: 0.5508 - learning_rate: 0.0010
Epoch 6/250
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.7318 - loss: 0.5466 - learning_rate: 0.0010
Epoch 7/250
1715/1715 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step 

In [28]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7302 - loss: 0.9165
Loss: 0.9164903163909912, Accuracy: 0.7301748991012573


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [34]:
#using robust scaler
from sklearn.preprocessing import  RobustScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [35]:
# Optimized Model
nn_optimized = tf.keras.models.Sequential()

# First hidden layer (Swish activation)
nn_optimized.add(tf.keras.layers.Dense(units=256, activation="swish", input_dim=X_train_scaled.shape[1]))

# Second hidden layer (SELU activation)
nn_optimized.add(tf.keras.layers.Dense(units=128, activation="selu"))

# Third hidden layer (More neurons)
nn_optimized.add(tf.keras.layers.Dense(units=64, activation="relu"))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile Model (Hinge loss)
nn_optimized.compile(loss="hinge", optimizer="nadam", metrics=["accuracy"])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
# Compute class weights to balance data
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
class_weights = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Train the optimized model
nn_optimized.fit(X_train_scaled, y_train, epochs=150, batch_size=32, class_weight=class_weights_dict)

Epoch 1/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6752 - loss: 0.8391
Epoch 2/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6915 - loss: 0.8173
Epoch 3/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6867 - loss: 0.8204
Epoch 4/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6748 - loss: 0.8249
Epoch 5/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6817 - loss: 0.8191
Epoch 6/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6695 - loss: 0.8222
Epoch 7/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6809 - loss: 0.8145
Epoch 8/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6770 - loss: 0.8244
Epoch 9/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6792 - loss: 0.8239
Epoch 10/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6842 - loss: 0.8205
Epoch 11/150
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6842 - loss: 0.8184
Epoch 12/150
751/751 ━━━━━━━━━━━━━━━━━━━━

In [39]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - 2ms/step - accuracy: 0.5731 - loss: 164104.2344
Loss: 164104.234375, Accuracy: 0.5730806589126587


In [40]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

In [41]:
from google.colab import files

# Download the saved model
files.download("AlphabetSoupCharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>